In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipywidgets.widgets as widgets
import time
import serial
from PIL import ImageColor
from IPython.display import display, HTML, IFrame
from aiy.vision.streaming.server import StreamingServer
from picamera import PiCamera
from robot import Robot

## Button Controller

Simple test to run the AIY robot using `ipywidgets` buttons. Creating serial connection to Pico, robot object and streaming server.

In [3]:
s = serial.Serial("/dev/ttyACM1", 115200)   # Note ACM0 is CIRCUITPY Drive

In [4]:
bot = Robot()

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


In [5]:
camera = PiCamera(sensor_mode=4, resolution=(410, 308))
server = StreamingServer(camera, bitrate=75000)

Now we create the buttons and add the callbacks.

In [6]:
button_layout = widgets.Layout(width='80px', height='60px', align_self='center')

stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
speed_slider = widgets.FloatSlider(value=0.5, min=0.2, max=1.0, step=0.1, description='Speed:',
                                   disabled=False, continuous_update=False, orientation='vertical',
                                   readout=True, readout_format='.1f')

# video frame
video = widgets.HTML(value="<iframe src='http://192.168.0.138:4664/' width=450 height=380>")

# lights
toggle = widgets.ToggleButton(description='Light', 
                              layout=widgets.Layout(width='80px', height='30px', margin='30px', align_self='center'))
color = widgets.ColorPicker(concise=False, description='Color', value='#FF0000', disabled=False)

# display buttons
light_box = widgets.VBox([toggle,color])
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button, light_box])

# all togehter
ui = widgets.HBox([video, controls_box, speed_slider])

In [7]:
def doStop(button):
    bot.stop()

def doForward(button):
    bot.forward(speed_slider.value)
    time.sleep(0.5)
    bot.stop()
    
def doBackward(button):
    bot.backward(speed_slider.value)
    time.sleep(0.5)
    bot.stop()
    
def doLeft(button):
    bot.left(speed_slider.value)
    time.sleep(0.5)
    bot.stop()
    
def doRight(button):
    bot.right(speed_slider.value)
    time.sleep(0.5)
    bot.stop()
    
def doSwitch(change):
    if(change['new']):
        c=ImageColor.getcolor(color.value, "RGB")
        command=bytes(f"neo {c[0]} {c[1]} {c[2]}\r", 'utf-8')
        s.write(command)
    else:
        s.write(b"neo 0 0 0\r")

stop_button.on_click(doStop)
forward_button.on_click(doForward)
backward_button.on_click(doBackward)
left_button.on_click(doLeft)
right_button.on_click(doRight)
toggle.observe(doSwitch, 'value')

Display the buttons and video feed.

In [8]:
display(ui)

### Control and enjoy

Note the poor little Pi Zero is kind of struggling and the resonse is sluggish. So drive slow 🙂 When done run the cells below to close everything properly. You still need to restart the kernel to run it again because of the Servos apparently not releasing the pins properly.

In [ ]:
# Cleanup
server.close()
camera.close()
bot.close()
s.close()